In [7]:
import pickle
from pathlib import Path

Path("temp.pickle").write_bytes(b"")

a = [1,2,[1,2,3,4]]
with open("temp.pickle", "ab") as f:
    pickle.dump(a, f)
    pickle.dump(a, f)
    
a = [1,2,[1,2,3,5]]
with open("temp.pickle", "ab") as f:
    pickle.dump(a, f)


b = []
with open("temp.pickle", "rb") as f:    
    while 1:
        try:
            b.append(pickle.load(f))
        except EOFError:
            break
print(b)

[[1, 2, [1, 2, 3, 4]], [1, 2, [1, 2, 3, 4]], [1, 2, [1, 2, 3, 5]]]


In [6]:
import datetime
import os
from pathlib import Path

import glob2
import numpy as np
import scipy.io.wavfile
from matplotlib import pyplot as plt
from scipy import signal
from tqdm import tqdm
import skimage
from scipy.signal import find_peaks

from utils.data_reading.catalogs.catalog import merge_catalogs
from utils.data_reading.catalogs.ISC import ISC_file
from utils.data_reading.catalogs.twin_catalog import TwinCatalog
from utils.data_reading.sound_data.station import StationsCatalog
from utils.physics.sound.sound_model import HomogeneousSoundModel
from utils.physics.sound.sound_velocity_grid import MonthlySoundVelocityGrid, MonthlySoundVelocityGridOptimized
from utils.physics.utils.generic import deg_to_m, euclidian_distance
from utils.transformations.features_extractor import STFTFeaturesExtractor

In [14]:
# input files
isc_dir = f"/home/plerolland/Bureau/catalogs/ISC"
catalog_path = "/media/plerolland/LaBoite/twin_catalog"
datasets_yaml = "/home/plerolland/Bureau/dataset.yaml"
out_dir = "/media/plerolland/LaBoite/associator"

DELTA = datetime.timedelta(seconds=200)
TIME_RES = 0.5
F_RES = 0.5
stft_computer = STFTFeaturesExtractor(None, vmin=60, vmax=140)

In [11]:
stations = StationsCatalog(datasets_yaml).filter_out_undated().filter_out_unlocated()
catalog = TwinCatalog(catalog_path, isc_dir, stations)

In [15]:
csv = "ID, station, date"

existing_stations = set()
for ID, twin_association in tqdm(catalog.items.items()):
    for event in twin_association.acoustic_association.events:
        station = event.station
        if station in existing_stations:
            for st in existing_stations:
                if st == station:
                    station = st
            existing_stations.add(station)
            
        stft_computer.manager = event.station.get_manager()
        if not (station.manager.dataset_start < event.date-DELTA < station.manager.dataset_end and station.manager.dataset_start < event.date+DELTA < station.manager.dataset_end):
            continue
        
        s_f = 250 if "IMS" in event.station.dataset else 240
        stft_computer.manager.cache_size = 0
        stft_computer.nperseg = int(stft_computer.manager.sampling_f / F_RES)
        stft_computer.overlap = 1 - TIME_RES * stft_computer.manager.sampling_f / stft_computer.nperseg
        
        file_name = f'{out_dir}/{station.name}/{event.date.strftime("%Y%m%d_%H%M%S")}.png'
        Path(f'{out_dir}/{station.name}').mkdir(parents=True, exist_ok=True)
        
        if not Path(file_name).exists():
            stft_features = stft_computer.get_features(event.date-DELTA, event.date+DELTA)
            stft_computer._save_features(stft_features, file_name)
        
        csv += f'\n{ID},{event.station.name},{event.date.strftime("%Y%m%d_%H%M%S")}'

 30%|███       | 3074/10134 [48:08<1:47:27,  1.09it/s]/home/plerolland/Bureau/OHASISBIO_dataset/src/utils/transformations/features_extractor.py:157: UserWarning: /media/plerolland/LaBoite/associator/H01W1/20200121_035610.png is a low contrast image
  skimage.io.imsave(path, spectro.astype(np.uint8))
100%|██████████| 10134/10134 [2:34:00<00:00,  1.10it/s] 


In [16]:
with open(f'{out_dir}/index.csv', 'w') as f:
    f.write(csv)